In [1]:
'''import sys
!{sys.executable} -m pip install BeautifulSoup4
!{sys.executable} -m pip install lxml
!{sys.executable} -m pip install html5lib
!{sys.executable} -m pip install requests
'''
#!/usr/bin/python
# -*- coding: UTF-8 -*-
import re
import csv
import time
import requests
from bs4 import BeautifulSoup
PTT_URL = 'https://www.ptt.cc'
target_url = '/bbs/MacShop/index.html'

    

def get_web_page(url):
    resp = requests.get(
        url=url,
        cookies={'over18': '1'}
    )
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
        return None
    else:
        return resp.text

def get_content(dom):
    soup = BeautifulSoup(dom, 'html.parser') 
    divs = soup.find_all('div', id='main-container')
    price = "none"
    for d in divs:
        content=re.split('：| |\n',d.text)
        #print(content)
        for index in range(len(content)):
            if content[index] =="[交易價格]":
                pre_price = content[index+1]
                price_list = filter(str.isdigit, pre_price)
                price = ''.join(list(price_list))
    return price

def get_articles(dom, date):
    soup = BeautifulSoup(dom, 'html.parser') 
    paging_div = soup.find('div', 'btn-group btn-group-paging')# 取得上一頁的連結
    prev_url = paging_div.find_all('a')[1]['href']
    articles = []  # 儲存取得的文章資料
    divs = soup.find_all('div', 'r-ent')
    for d in divs:
        D_day = d.find('div', 'date').text.strip()
        if D_day == date:  
            # 取得推文數
            push_count = 0
            push_str = d.find('div', 'nrec').text
            if push_str:
                try:
                    push_count = int(push_str)
                except ValueError:
                    if push_str == '爆':
                        push_count = 99
                    elif push_str.startswith('X'):
                        push_count = -10
            # 取得文章連結及標題
            if d.find('a'):  # 有超連結，表示文章存在，未被刪除
                href = d.find('a')['href']
                price = get_content(get_web_page(PTT_URL + href))
                title = d.find('a').text
                author = d.find('div', 'author').text if d.find('div', 'author') else ''
                titles = title.split("]") 
                #print(titles)
                if len(titles)==2:
                    articles.append({
                        'category':titles[0][1:3],
                        'title': titles[1][1:],
                        'href': href,
                        'push_count': push_count,
                        'author': author,
                        'price' : price,
                        'date' : D_day
                    })
    return articles, prev_url

    
if __name__ == '__main__':
    articles,detail = [],[]
    new_articles = 1
    date = time.strftime("%m/%d").lstrip('0')
    while new_articles:
        if new_articles!=1 :
            articles += new_articles
        current_page = get_web_page(PTT_URL + target_url)
        if current_page:
            new_articles, target_url = get_articles(current_page, date)
    with open('ptt.csv','w',newline='') as csvfile:
        writer = csv.writer(csvfile)
        fieldnames = ['category', 'title', 'href','push_count','author','price','date']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for article in articles:
            print(article)
            writer.writerow(article)

    #for a in articles:
        #print(a)


{'category': '販售', 'title': '全新19年 MacBook Air 13"1.6/8/256 現貨', 'href': '/bbs/MacShop/M.1567845026.A.015.html', 'push_count': 0, 'author': 'coffeeyuan', 'price': '39900', 'date': '9/07'}
{'category': '收購', 'title': '雙北 徵求iphone7 32G', 'href': '/bbs/MacShop/M.1567845516.A.259.html', 'push_count': 0, 'author': 'supremekuo', 'price': 'none', 'date': '9/07'}
{'category': '販售', 'title': 'iPad Pro 12.9 第三代 wifi 256GB  ', 'href': '/bbs/MacShop/M.1567846067.A.65D.html', 'push_count': 0, 'author': 'akashi998', 'price': '30000', 'date': '9/07'}
{'category': '販售', 'title': '台北 Macbook Air 256GB 2016年出產', 'href': '/bbs/MacShop/M.1567846147.A.455.html', 'push_count': 0, 'author': 'reticentmind', 'price': '17000', 'date': '9/07'}
{'category': '販售', 'title': '桃園 Iphone8 64G 黑 9成新', 'href': '/bbs/MacShop/M.1567846372.A.E13.html', 'push_count': 0, 'author': 'akr3998', 'price': '11500', 'date': '9/07'}
{'category': '販售', 'title': '桃園 全新 iPhone8 64G 金', 'href': '/bbs/MacShop/M.1567846408.A.D82.html', 'p